In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [13]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_rubert_from_discriminator'
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [10]:
import pandas as pd

discr_df = pd.read_csv(pathjoin(DATA_DIR, 'min_gpt_bpe', 'ru_one_dataset'))
discr_df['target'] = 'gen'
discr_df = discr_df.append(pd.read_csv(pathjoin(DATA_DIR, 'ru_train')))
discr_df.to_csv(pathjoin(DATA_DIR, 'ru_discr.csv'))
discr_df.head()

,Unnamed: 0,target,text
0,0,gen,"глава ii . и все же , конечно , были предчу..."
1,1,gen,глава xvii : уаи обеспечивает в 15 - 74 22 . ...
2,2,gen,"глава xvii : "" вот , дети , наверное , и друг..."
3,3,gen,"глава xvii : братья и сестры , дети , которые..."
4,4,gen,ibc 1 1 - ds - oil # < p > 192 < br / > наиди...


In [11]:
transformer_model = 'DeepPavlov/rubert-base-cased'
MAX_TOKENS = 512

In [12]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "ru_discr.csv"), 
    pathjoin(DATA_DIR, "ru_test"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data
<class 'data_processing.ClassificationDatasetReader'> /home/mlepekhin/data/ru_discr.csv



Building the vocabulary


In [14]:
model = build_transformer_model(vocab, transformer_model)

Building the model


In [15]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [16]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [17]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training



Finished training


In [18]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

total 681M
-rw-rw-r-- 1 mlepekhin mlepekhin 681M окт  1 17:04 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K окт  1 16:59 log
